In [8]:
%matplotlib inline
import pandas as pd
import geopandas as gp
import numpy as np
import os

PARQA = os.getenv('PARQA')

In [6]:
ontology = pd.read_csv(PARQA + 'parqa/311/ONTOLOGY/ontology/Ontology_verified.csv')[['cleanName','SIGNNAME','Type']]

In [7]:
ontology.head(2)

,cleanName,SIGNNAME,Type
0,geo soilan park - battery park city,battery park city,other
1,brookville park,brookville park,park_direct


## I. Ontology to DPR property data

In [9]:
ontology['pDistrict'] = np.nan

In [10]:
ontology.Type.value_counts()

park_direct    714
pgs            373
other          285
empiric        209
pool            60
beach           12
golf            12
school           9
recr             4
Name: Type, dtype: int64

In [18]:
## data for all types of keys in ontology
dfs = {'park_direct': gp.read_file(PARQA + 'data/DPR_property/csv_ll_pD/' + 'parks_pd_ll.geojson' ),
       'pgs': gp.read_file(PARQA + 'data/DPR_property/csv_ll_pD/' + 'playground_pd_ll.geojson' ),
       'other': gp.read_file(PARQA + 'data/DPR_property/csv_ll_pD/' + 'parks_pd_ll.geojson' ),
       'empiric': gp.read_file(PARQA + 'data/DPR_property/csv_ll_pD/' + 'parks_pd_ll.geojson' ),
       'pool': gp.read_file(PARQA + 'data/DPR_property/csv_ll_pD/' + 'pools_pd_ll.geojson' ),
       'beach': gp.read_file(PARQA + 'data/DPR_property/csv_ll_pD/' + 'beaches_pd_ll.geojson' ),
       'golf': gp.read_file(PARQA + 'data/DPR_property/csv_ll_pD/' + 'golf_courses_pd_ll.geojson' ),
       'school':gp.read_file(PARQA + 'data/DPR_property/csv_ll_pD/' + 'ps_pd_ll.geojson' ),
       'recr': gp.read_file(PARQA + 'data/DPR_property/csv_ll_pD/' + 'recr_c_pd_ll.geojson' )}

In [19]:
for key in dfs:
    print key, len(dfs[key][pd.isnull(dfs[key].parkDistrict)])

school 0
other 40
empiric 40
pgs 0
recr 0
golf 0
beach 0
pool 1
park_direct 40


In [22]:
# rename all namincolumns to NAME
dfs['school'] = dfs['school'].rename(columns={'PSID':'NAME'})
dfs['other'] = dfs['other'].rename(columns={'SIGNNAME':'NAME'})
dfs['empiric'] = dfs['empiric'].rename(columns={'SIGNNAME':'NAME'})
dfs['pgs'] = dfs['school'].rename(columns={'Name':'NAME'})
dfs['beach'] = dfs['beach'].rename(columns={'name':'NAME'})
dfs['pool'] = dfs['pool'].rename(columns={'Name':'NAME'})
dfs['park_direct'] = dfs['park_direct'].rename(columns={'SIGNNAME':'NAME'})

for key in dfs:
    if 'NAME'not in dfs[key].columns:
        print key,ddfs[key].columns

In [26]:
for item in dfs.values():
    item['NAME'] = item.NAME.str.lower()

In [25]:
ontology.ix[ontology.Type=='pgs', :].merge( dfs['pgs'][['NAME'],''], how='left')

,cleanName,SIGNNAME,Type,pDistrict
9,col david marcus playground,Col David Marcus Mem Playground,pgs,NaN
13,mccarren park - vincent v abate playground,Vincent V Abate Playground,pgs,NaN
24,harold l ickes playground,Harold Ickes Playground,pgs,NaN
27,hoover manton playground,Manton Playground,pgs,NaN
33,emerald playground - ps 200,Emerald Playground,pgs,NaN
